# Text data generalization

## 1. 读取推文信息，生成用于时间主题分析数据集


In [ ]:
import pandas as pd
import json
import os
os.chdir("D:/uppsala/16. data of social mining/data")

# 文件名为 'tweets.dat'
file_path = 'tweets.dat'

# 存储提取出的推文数据的列表
tweet_records = []

try:
    with open(file_path, 'r', encoding='utf-8') as f:
        # 逐行读取文件
        for line in f:
            # 1. 解析每一行为一个 JSON 对象
            try:
                tweet_json = json.loads(line.strip())
            except json.JSONDecodeError:
                # 简单地跳过格式错误的行
                print(f"Skipping badly formatted line: {line[:50]}...")
                continue

            # 2. 提取所需的字段
            tweet_id = tweet_json.get('id')
            author_id = tweet_json.get('author_id')
            text = tweet_json.get('text')
            created_at_str = tweet_json.get('created_at') # 格式如 "2015-12-12T23:59:59.000Z"
            language = tweet_json.get("lang")
            metrics = tweet_json.get("public_metrics", {})
            retweet_count = metrics.get("retweet_count", 0)
            favorite_count = metrics.get("like_count", 0)
           
            # 3. 提取日期部分 (YYYY-MM-DD)
            # 仅取 'T' 之前的部分即为日期
            posting_date = created_at_str.split('T')[0] if created_at_str else None

            # 收集数据
            tweet_records.append({
                'tweet_id': tweet_id,
                'author_id': author_id,
                'tweet_text': text,
                'posting_date': posting_date,
                'language': language,
                'retweet_count': retweet_count,
                'favorite_count': favorite_count
            })

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
    # 如果文件不存在，您可以跳过这一步或加载一个模拟数据
    # return None

# 4. 转换为 Pandas DataFrame
tweets= pd.DataFrame(tweet_records)

# 查看结果
print("--- 推文数据集 ---")
print(tweets.head())
print(f"\n总记录数: {len(tweets)}")


In [ ]:
print(tweets['posting_date'].value_counts().sort_index())
total_tweets = tweets['posting_date'].value_counts().sum()
print(f"总推文数量: {total_tweets}")

## 2.生成推文 + 账户信息的数据集

* 为推文添加发布者的账户的信息：语言（language）、类型（type）和立场（stance）


In [ ]:
accounts = pd.read_csv("accounts.tsv", sep="\t", dtype={"author_id": str})
# 重命名 accounts的 Lang 字段为 account_language，和 tweets 数据框中的 language 以示区别

accounts.rename(columns={"Lang": "account_language"}, inplace=True)

print("✅ 已载入  accounts.tsv")
print( accounts.head(5))
print( accounts.shape)


In [ ]:

# === 5. 合并数据 ===
tweets_with_accounts = pd.merge(
    tweets,
    accounts,
    on="author_id",  # 两边列名相同，直接用 on
    how="inner"
)
print("--- 合并后的数据集 ---")
print(tweets_with_accounts.head())
print(f"\n总记录数: {len(tweets_with_accounts)}")

In [ ]:
tweets_with_accounts.to_csv("tweets_with_accounts.csv", index=False,encoding="utf-8-sig")

In [ ]:
print(tweets_with_accounts['posting_date'].value_counts().sort_index())
total_tweets_accounts = tweets_with_accounts['posting_date'].value_counts().sum()
print(f"推文账户合并信息: {total_tweets_accounts}")